In [1]:
pwd

'C:\\Users\\sidha\\Downloads'

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('train2.csv')

In [4]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Deck,FamilySize,FsizeD,NameLength,NlengthD,Title,TicketNumber
0,1,0,3,"Braund, Mr. Owen Harris",1,-0.560929,1,0,A/5 21171,-0.502445,NaN,2,8,2,2,23,2,2,21171.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,0.602418,1,0,PC 17599,0.786845,C85,0,2,2,2,51,0,3,17599.0
2,3,1,3,"Heikkinen, Miss. Laina",0,-0.270092,0,0,STON/O2. 3101282,-0.488854,NaN,2,8,1,1,22,2,1,3101282.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,0.384291,1,0,113803,0.420730,C123,2,2,2,2,44,0,3,113803.0
4,5,0,3,"Allen, Mr. William Henry",1,0.384291,0,0,373450,-0.486337,NaN,2,8,1,1,24,2,2,373450.0


In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
# The columns we'll use to predict the target
predictors = ["Pclass", "Sex", "Age","SibSp", "Parch", "Fare",
              "Embarked","NlengthD", "FsizeD", "Title","Deck"]
target="Survived"
# Initialize our algorithm class
alg = LinearRegression()

# Generate cross validation folds
kf = KFold(n_splits=3, random_state=1)

predictions = []

In [6]:
for train, test in kf.split(df):
    train_predictors = (df[predictors].iloc[train,:])
    train_target = df[target].iloc[train]
    # Put the dataset in the model
    alg.fit(train_predictors, train_target)
    # Predictions on the test fold using the fitted model
    test_predictions = alg.predict(df[predictors].iloc[test,:])
    predictions.append(test_predictions)

In [7]:
import math
predictions = np.concatenate(predictions, axis=0)

predictions_list = np.array(predictions)

for i in range(0,len(predictions_list)):
    predictions_list[i] = 1 / (1 + math.exp(-1 * predictions_list[i]))

match = 0
unmatch = 0

for i in range(0,len(predictions_list)):
    if(df['Survived'].iloc[i] == 0):
        if(predictions_list[i] < 0.5):
            match = match+1
        else:
            unmatch = unmatch +1
    else:
        if(predictions_list[i] > 0):
            match = match+1
        else:
            unmatch = unmatch +1

accuracy = match/(match+unmatch)
print(accuracy)

0.42536475869809204


In [8]:
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit

predictors = ["Pclass", "Sex", "Fare", "Embarked","Deck","Age",
              "FsizeD", "NlengthD","Title","Parch"]

lr = LogisticRegression(random_state=1)
cv = ShuffleSplit(n_splits=10, test_size=0.3, random_state=50)

#Accuracy on all cross validation folds
scores = cross_val_score(lr, df[predictors], 
                                          df["Survived"],scoring='f1', cv=cv)
# Mean of all the results for every fold
print(scores.mean())

0.7623961181177553


C:\Users\sidha\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\sidha\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\sidha\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\sidha\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\sidha\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence t

In [9]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_predict

import numpy as np
predictors = ["Pclass", "Sex", "Age",
              "Fare","NlengthD","NameLength", "FsizeD", "Title","Deck"]

rf = RandomForestClassifier(random_state=1, n_estimators=10, min_samples_split=2, 
                            min_samples_leaf=1)
kf = KFold( n_splits=5, random_state=1)
cv = ShuffleSplit(n_splits=10, test_size=0.3, random_state=50)

predictions = cross_val_predict(rf, df[predictors],df["Survived"],cv=kf)
predictions = pd.Series(predictions)
scores = cross_val_score(rf, df[predictors], df["Survived"],
                                          scoring='f1', cv=kf)
print(scores.mean())

0.7463882768106281
